# Comparing Transformer and GATr Models for Convex Hull Volume Regression

In this notebook, we will perform a surrogate regression task to infer the volume of a 5-point convex hull in 3D space. We will train two models:

1. **Regular Transformer**: A standard transformer model to learn the relationship between the point cloud shapes and their corresponding volumes.
2. **GATr (Geometric Algebra Transformer)**: A specialized transformer model leveraging geometric algebra for enhanced performance on geometric data.

We will compare these models in terms of training efficiency, loss, and other relevant metrics.

## Table of Contents

1. [Setup and Imports](#setup-and-imports)
2. [Data Loading and Preprocessing](#data-loading-and-preprocessing)
3. [Dataset and DataLoader Definition](#dataset-and-dataloader-definition)
4. [Regular Transformer Model](#regular-transformer-model)
5. [Training Loop for Transformer](#training-loop-for-transformer)
6. [GATr Model](#gatr-model)
7. [Training Loop for GATr](#training-loop-for-gatr)
8. [Evaluation and Comparison](#evaluation-and-comparison)      

## Setup and Imports
First, we need to import the necessary libraries and set up the environment.

In [1]:
# setup.ipynb

import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch_geometric as pyg
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from tqdm import tqdm
from lab_gatr import PointCloudPoolingScales, LaBGATr
import matplotlib.pyplot as plt

# Ensure reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Check for GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


/home/ajay/anaconda3/envs/anatomygen_gatr/lib/python3.10/site-packages/gatr/primitives/bilinear.py:38: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  sparse_basis = torch.loa

## Data Loading and Preprocessing

We will load the convex hull data from the `3d_point_cloud_dataset` directory. Each file represents a convex hull with 5 points in 3D space. We will compute the volume of each convex hull using the points.

**Note**: Ensure that the `3d_point_cloud_dataset` directory is in the current working directory and contains 5000 sample files named as `convex_hull_0.txt`, `convex_hull_1.txt`, ..., `convex_hull_4999.txt`.

In [2]:
# data_loading.ipynb

import os
from scipy.spatial import ConvexHull

class ConvexHullDataset(Dataset):
    def __init__(self, data_dir):
        self.data_dir = data_dir
        self.file_names = sorted([f for f in os.listdir(data_dir) if f.endswith('.txt')])
        self.samples = []
        self._prepare_dataset()
    
    def _prepare_dataset(self):
        for file_name in tqdm(self.file_names, desc="Loading data"):
            file_path = os.path.join(self.data_dir, file_name)
            with open(file_path, 'r') as f:
                lines = f.readlines()[1:]  # Skip header
                points = []
                for line in lines:
                    x, y, z = map(float, line.strip().split())
                    points.append([x, y, z])
                points = np.array(points)
                if points.shape[0] < 4:
                    # Convex hull in 3D requires at least 4 non-coplanar points
                    volume = 0.0
                else:
                    try:
                        hull = ConvexHull(points)
                        volume = hull.volume
                    except:
                        # In case points are coplanar or singular
                        volume = 0.0
                self.samples.append({'points': points, 'volume': volume})
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        sample = self.samples[idx]
        points = sample['points']
        volume = sample['volume']
        return {'points': torch.tensor(points, dtype=torch.float32), 'volume': torch.tensor(volume, dtype=torch.float32)}

### Splitting the Data

We will split the data into training and testing sets with an 80-20 split.

In [3]:
# data_splitting.ipynb

data_dir = '3d_point_cloud_dataset'  # Adjust the path if necessary
dataset = ConvexHullDataset(data_dir)

train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size

train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

print(f"Training samples: {len(train_dataset)}")
print(f"Testing samples: {len(test_dataset)}")

Loading data:   0%|          | 0/5000 [00:00<?, ?it/s]

Loading data: 100%|██████████| 5000/5000 [00:00<00:00, 5428.80it/s]

Training samples: 4000
Testing samples: 1000


## Dataset and DataLoader Definition

We will define a custom collate function to handle batching of point clouds with varying points if necessary. However, since each convex hull has exactly 5 points, we can use standard batching.

In [4]:
# dataloader_definition.ipynb

def collate_fn(batch):
    points = torch.stack([item['points'] for item in batch], dim=0)  # Shape: [batch_size, 5, 3]
    volumes = torch.stack([item['volume'] for item in batch], dim=0)  # Shape: [batch_size]
    return points, volumes

batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

## Regular Transformer Model

We will define a simple transformer-based regression model. The model will take the 5 points (each with 3 coordinates) as input and output the predicted volume.

### Model Architecture

- **Input Embedding**: Linear layer to project the 3D coordinates to a higher-dimensional space.
- **Transformer Encoder**: Processes the embedded points.
- **Pooling**: Global average pooling to aggregate information from all points.
- **Regression Head**: Linear layers to predict the volume.

### Implementation

In [5]:
# transformer_model.ipynb

class TransformerRegressor(nn.Module):
    def __init__(self, input_dim=3, embed_dim=64, num_heads=8, num_layers=3, dropout=0.1):
        super(TransformerRegressor, self).__init__()
        self.embedding = nn.Linear(input_dim, embed_dim)
        encoder_layer = nn.TransformerEncoderLayer(d_model=embed_dim, nhead=num_heads, dropout=dropout)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.pooling = nn.AdaptiveAvgPool1d(1)
        self.regressor = nn.Sequential(
            nn.Linear(embed_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 1)
        )
    
    def forward(self, x):
        """
        x: [batch_size, num_points, 3]
        """
        x = self.embedding(x)  # [batch_size, num_points, embed_dim]
        x = x.permute(1, 0, 2)  # [num_points, batch_size, embed_dim] for Transformer
        x = self.transformer(x)  # [num_points, batch_size, embed_dim]
        x = x.permute(1, 2, 0)  # [batch_size, embed_dim, num_points]
        x = self.pooling(x).squeeze(-1)  # [batch_size, embed_dim]
        x = self.regressor(x).squeeze(-1)  # [batch_size]
        return x

## Training Loop for Transformer

We will define the training and evaluation loops for the transformer model.

In [6]:
# transformer_training.ipynb

def train_model(model, train_loader, test_loader, epochs=50, lr=1e-3):
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    train_losses = []
    test_losses = []
    
    for epoch in range(1, epochs + 1):
        model.train()
        running_loss = 0.0
        for points, volumes in train_loader:
            points = points.to(device)
            volumes = volumes.to(device)
            
            optimizer.zero_grad()
            outputs = model(points)
            loss = criterion(outputs, volumes)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item() * points.size(0)
        
        epoch_loss = running_loss / len(train_loader.dataset)
        train_losses.append(epoch_loss)
        
        # Evaluation
        model.eval()
        test_loss = 0.0
        with torch.no_grad():
            for points, volumes in test_loader:
                points = points.to(device)
                volumes = volumes.to(device)
                outputs = model(points)
                loss = criterion(outputs, volumes)
                test_loss += loss.item() * points.size(0)
        test_loss /= len(test_loader.dataset)
        test_losses.append(test_loss)
        
        print(f"Epoch {epoch}/{epochs} - Train Loss: {epoch_loss:.4f}, Test Loss: {test_loss:.4f}")
    
    return train_losses, test_losses

# Initialize model
transformer_model = TransformerRegressor().to(device)

# Train the model
transformer_train_losses, transformer_test_losses = train_model(transformer_model, train_loader, test_loader, epochs=50, lr=1e-3)

/home/ajay/anaconda3/envs/anatomygen_gatr/lib/python3.10/site-packages/torch/nn/modules/transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


Epoch 1/50 - Train Loss: 871.7245, Test Loss: 617.0840
Epoch 2/50 - Train Loss: 578.2365, Test Loss: 618.8878
Epoch 3/50 - Train Loss: 579.3415, Test Loss: 615.7886
Epoch 4/50 - Train Loss: 579.5453, Test Loss: 576.6891
Epoch 5/50 - Train Loss: 585.1776, Test Loss: 617.2463
Epoch 6/50 - Train Loss: 586.3466, Test Loss: 616.9638
Epoch 7/50 - Train Loss: 586.1505, Test Loss: 618.3257
Epoch 8/50 - Train Loss: 585.4103, Test Loss: 616.5834
Epoch 9/50 - Train Loss: 583.9380, Test Loss: 576.2822
Epoch 10/50 - Train Loss: 540.5633, Test Loss: 578.6763
Epoch 11/50 - Train Loss: 507.3119, Test Loss: 514.7278
Epoch 12/50 - Train Loss: 481.2433, Test Loss: 478.3207
Epoch 13/50 - Train Loss: 464.3097, Test Loss: 472.5044
Epoch 14/50 - Train Loss: 444.4343, Test Loss: 435.4570
Epoch 15/50 - Train Loss: 433.9258, Test Loss: 444.6836
Epoch 16/50 - Train Loss: 409.0888, Test Loss: 395.4681
Epoch 17/50 - Train Loss: 411.4604, Test Loss: 371.8771
Epoch 18/50 - Train Loss: 368.7528, Test Loss: 366.7306
E

## GATR MODEL

In [20]:
import torch
from lab_gatr import PointCloudPoolingScales, LaBGATr
import torch_geometric as pyg
from gatr.interface import embed_oriented_plane, extract_translation

n = 1000

pos, orientation = torch.rand((n, 3)), torch.rand((n, 3))
scalar_feature = torch.rand(n)

transform = PointCloudPoolingScales(rel_sampling_ratios=(0.2,), interp_simplex='triangle')
dummy_data = transform(pyg.data.Data(pos=pos, orientation=orientation, scalar_feature=scalar_feature))

class GeometricAlgebraInterface:
    num_input_channels = num_output_channels = 1
    num_input_scalars = num_output_scalars = 1

    @staticmethod
    @torch.no_grad()
    def embed(data):

        multivectors = embed_oriented_plane(normal=data.orientation, position=data.pos).view(-1, 1, 16)
        scalars = data.scalar_feature.view(-1, 1)

        return multivectors, scalars

    @staticmethod
    def dislodge(multivectors, scalars):
        return extract_translation(multivectors).squeeze()
    
model = LaBGATr(GeometricAlgebraInterface, d_model=8, num_blocks=10, num_attn_heads=4, use_class_token=False)
output = model(dummy_data)
print(output.shape)

gatr_train_losses, gatr_test_losses = train_model(model, train_loader, test_loader, epochs=50, lr=1e-3)


LaB-GATr (261761 parameters)
torch.Size([1000, 3])


/tmp/ipykernel_1031615/1057448476.py:29: GATrDeprecationWarning: The function "extract_translation" is deprecated, because it is not equivariant.
  return extract_translation(multivectors).squeeze()


AttributeError: 'Tensor' object has no attribute 'orientation'